In [175]:
import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [176]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [177]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [178]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
from torchsummary import summary
for p in model.parameters():
    p = torch.tensor(np.random.normal(0,0.1,p.size()))
summary(model.to('cuda'), (1,28,28))

Using cuda device
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
           Flatten-1                  [-1, 784]               0
            Linear-2                  [-1, 512]         401,920
              ReLU-3                  [-1, 512]               0
            Linear-4                  [-1, 512]         262,656
              ReLU-5                  [-1, 512]               0
            Linear-6                   [-1, 10]           5,130
Total params: 669,706
Trainable params: 669,706
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.02
Params size (MB): 2.55
Estimated Total Size (MB): 2.58
----------------------------------------------------------------


In [179]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

In [180]:
def train(dataloader, model, loss_fn, optimizer):
    loss_list = []
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)
        loss_list.append(loss.item())
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {np.mean(loss_list):>7f}  [{current:>5d}/{size:>5d}]")

In [181]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [182]:
epochs = 8
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
print("Done!")

Epoch 1
-------------------------------
loss: 2.314523  [    0/60000]
loss: 1.488858  [ 6400/60000]
loss: 1.120572  [12800/60000]
loss: 0.955373  [19200/60000]
loss: 0.862698  [25600/60000]
loss: 0.798081  [32000/60000]
loss: 0.750142  [38400/60000]
loss: 0.712904  [44800/60000]
loss: 0.685233  [51200/60000]
loss: 0.662079  [57600/60000]
Epoch 2
-------------------------------
loss: 0.346890  [    0/60000]
loss: 0.445479  [ 6400/60000]
loss: 0.443890  [12800/60000]
loss: 0.440897  [19200/60000]
loss: 0.437576  [25600/60000]
loss: 0.432765  [32000/60000]
loss: 0.429694  [38400/60000]
loss: 0.425733  [44800/60000]
loss: 0.423639  [51200/60000]
loss: 0.422211  [57600/60000]
Epoch 3
-------------------------------
loss: 0.296044  [    0/60000]
loss: 0.383469  [ 6400/60000]
loss: 0.379660  [12800/60000]
loss: 0.380476  [19200/60000]
loss: 0.379969  [25600/60000]
loss: 0.376340  [32000/60000]
loss: 0.374828  [38400/60000]
loss: 0.372725  [44800/60000]
loss: 0.371879  [51200/60000]
loss: 0.37

In [183]:
#test(test_dataloader, model, loss_fn)

In [184]:
# TRAIN WITH PSO

In [185]:
from Optimizer.optimizer import Optimizer
model = NeuralNetwork().to('cuda')
optimizer = Optimizer(model, 30, inertial_weight=0.9, self_conf=0.01, swarm_conf=1, norm=True)
def train(dataloader, model, loss_fn, optimizer):
    optimizer.epoch()
    model.train()
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        # Compute prediction error
        best_val = optimizer.step(X, y, loss_fn)

        if batch % 100 == 0:
            loss, current = best_val, batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [186]:
epochs = 2
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)

print("Done!")

Epoch 1
-------------------------------
loss: 3.805709  [    0/60000]
loss: 1.268615  [ 6400/60000]
loss: 0.991739  [12800/60000]
loss: 0.871994  [19200/60000]
loss: 0.804148  [25600/60000]
loss: 0.754171  [32000/60000]
loss: 0.716750  [38400/60000]
loss: 0.689124  [44800/60000]
loss: 0.666710  [51200/60000]
loss: 0.648675  [57600/60000]
Epoch 2
-------------------------------
loss: 0.290570  [    0/60000]
loss: 0.290570  [ 6400/60000]
loss: 0.290570  [12800/60000]
loss: 0.290570  [19200/60000]
loss: 0.290570  [25600/60000]
loss: 0.290570  [32000/60000]
loss: 0.290570  [38400/60000]
loss: 0.290570  [44800/60000]
loss: 0.290570  [51200/60000]
loss: 0.290570  [57600/60000]
Done!


In [187]:
test(test_dataloader, model, loss_fn)

Test Error: 
 Accuracy: 81.8%, Avg loss: 0.526548 

